In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import STL
from sklearn.metrics import  mean_squared_error
from sklearn.model_selection import GridSearchCV

from xgboost import XGBRegressor

from utils import *
from gradient import *

import warnings
warnings.filterwarnings("ignore")

In [ ]:
#DATA PREPROCESSING

data = pd.read_csv('./data/weather.csv')

#only date without time
data['date'] = data['date'].astype(str).str.slice(0, 10)
data['date'] = pd.to_datetime(data['date'])
data['precipitation'] = data['rain_1h'].fillna(0) + data['rain_3h'].fillna(0)


start_date = '2021-01-01'
data = data[data['date']>=start_date]

# print(check_for_missing_values(data))

#drop irrelevant and empty columns
data = data.drop(['dt','weather_main','city_name','lat','lon','feels_like','temp_min','temp_max',
                    'timezone','sea_level','grnd_level','wind_gust','weather_id','weather_icon',
                    'weather_description','snow_3h','snow_1h','rain_1h','rain_3h','visibility'], axis=1)

#group by date to get daily average values
data = data.groupby('date').agg(
    temp=('temp','mean'),
    pressure=('pressure', 'mean'),
    humidity=('humidity', 'mean'),
    wind_speed=('wind_speed', 'mean'),
    wind_deg=('wind_deg', 'mean'),
    clouds_all=('clouds_all', 'mean'),
    precipitation=('precipitation', 'sum'),
    dew_point=('dew_point', 'mean')
)

data = data.round(1)
data = data.reset_index()

# print(check_for_missing_values(data))

#data split for XGBoost
split = int(len(data) * 0.7)
trainxg = data[:split].copy()
valxg = data[split:].copy()
X_train = trainxg.drop(['precipitation','date'], axis=1)
y_train = trainxg['precipitation']
X_val = valxg.drop(['precipitation','date'], axis=1)
y_val = valxg['precipitation']

#data split for ARIMA
data = data.set_index('date')
val = data[split:].copy()
train = data[:split].copy()

#visualize data
plt.plot(train['precipitation'], color='b', linewidth=0.5, alpha=0.5, label='train')
plt.plot(val['precipitation'], color='black', linewidth=0.5, alpha=0.5, label='val')
plt.title('Precipitation')
plt.legend()
plt.show()

#stationarity check with augmented dickey fuller test
p_value = adfuller(train['precipitation'].dropna())[1]
if p_value <= 0.05: print('stationary')
else: print('not stationary')

data.head()

In [ ]:
plot_pacf(train['precipitation'], lags=20, method='ols')
plt.show()
plot_acf(train['precipitation'], lags=20)
plt.show()

stl = STL(train['precipitation'], period=365).fit()
stl.plot().show()

In [ ]:
#TIME SERIES ARIMA MODEL
p,d,q = 1,2,1
#WALK FORWARD EVALUATION
train.index = pd.DatetimeIndex(train.index.values, freq='D')
val.index = pd.DatetimeIndex(val.index.values, freq='D')
actual = val['precipitation']
wf = walk_forward_loop(train,val,column_name='precipitation',order=(p,d,q))
print('RMSE: ', np.sqrt(mean_squared_error(actual, wf)))

#arima visualization
plt.plot(train['precipitation'], color='b', linewidth=1, alpha=0.5, label='train')
plt.plot(val['precipitation'], color='mediumblue', linewidth=1, alpha=0.5, label='val')
plt.plot(wf, color='darkorange',linewidth=1, label='ARIMA model prediction')
plt.show()

In [ ]:
import requests
from dotenv import load_dotenv
import os
from datetime import date

key = os.getenv('api_key')

url = "http://api.weatherapi.com/v1/history.json"
params = {
    "key": key,
    "q": "Novi Sad", 
    "dt": date.today()
}
response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()
else:
    print(f"Request failed with status code {response.status_code}")

#Testing the model on newest data from the API
precip = data['forecast']['forecastday'][0]['day']['totalprecip_mm']
new_data = handle_data(data,train.columns)
wf = walk_forward_loop(train,new_data,column_name='precipitation',order=(p,d,q))

print('Predicted precipitation: ', 0 if wf[-1] < 0 else wf[-1])
print(f'Forecasted by weatherapi: {precip} mm')

In [ ]:
#GRADIENT BOOSTING FROM XGBOOST
# Find the best hyperparameters using GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6]
}
model = XGBRegressor(objective='reg:squarederror',random_state=42)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_

# Fit the model with the best hyperparameters
model = XGBRegressor(objective='reg:squarederror', n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], max_depth=best_params['max_depth'], random_state=42)
model.fit(X_train, y_train)   
y_pred = model.predict(X_val)
y_pred = np.maximum(y_pred, 0)

#xgboost visualization
plt.plot(trainxg['precipitation'], color='b', linewidth=1, alpha=0.5, label='train')
plt.plot(valxg['precipitation'], color='mediumblue', linewidth=1, alpha=0.5, label='val')
plt.plot(range(split,split+len(valxg)),y_pred, color='darkorange',linewidth=1, label='XGBoost prediction')
RMSE = np.sqrt(mean_squared_error(y_val, y_pred))
print(f'RMSE with XGBOOST: {RMSE}')

In [ ]:
#GRADIENT BOOSTING
#convert into numpy arrays and scale
y_train1 = np.array(y_train).reshape(X_train.shape[0],1)
y_val1 = np.array(y_val).reshape(X_val.shape[0],1)

#custom gradient boosting model
G = GradientBooster(max_depth=5,lr=0.1,iter=500,random_state=42)
models, losses, pred = G.train(X_train,y_train1)
y_pred1 = G.predict(models,X_val)
y_pred1 = np.maximum(y_pred1, 0)

#visualization
plt.plot(trainxg['precipitation'], color='b', linewidth=1, alpha=0.5, label='train')
plt.plot(valxg['precipitation'], color='mediumblue', linewidth=1, alpha=0.5, label='val')
plt.plot(range(split,split+len(valxg)),y_pred1, color='darkorange',linewidth=1, label='XGBoost prediction')

rmse = np.sqrt(mean_squared_error(y_val1, y_pred1))
print(f'RMSE with custom GradientBoost: {rmse}')